# 方向

1. 获取试卷类型
2. 根据试卷类型提取题目和答案

## 暂时规定的类型（可逐步添加-后期也会根据试卷类型进行对齐）：

### 0. 未知的

### 1. 高考试卷/标准试卷
此类试卷是标准的考试试卷类型
例如：
```
试卷部分：
一、
1....
二、
13....
三、
16....

答案部分：
同上试卷部分...
```

### 2. 试卷解析
此类试卷是讲解试卷的类型，每道题之中存在答案
例如：
```
试卷部分：
一、
1. 题干... 【答案】... 【解析】... 2....
二、
13. 题干...【答案】... 【解析】... 14....
三、
17. 题干...【答案】... 【解析】... 18....
```


## 加载一些可能会用到的函数

In [2]:
import sys

sys.path.append('../')

from alignment_utils import one_file_per_process
from alignment_utils import extract_and_combine_numbers
from alignment_utils import extract_and_combine_numbers_in_not_start
from alignment_utils import longest_increasing_subsequence_index
from alignment_utils import find_answer_split_str
from alignment_utils import find_next_question_index
from alignment_utils import refine_answers
from alignment_utils import match_specific_from_end
from alignment_utils import answer_area_str_process
from alignment_utils import generate_answer_area_string
from alignment_utils import align_answers_in_questions
from alignment_utils import match_specific_from_start
from alignment_utils import type_of_judgment
from alignment_utils import split_question
from alignment_utils import find_continuous_sequence
from alignment_utils import extract_and_combine_numbers_in_not_start_by_number

## Load data

In [3]:
import os
import glob
from pathlib import Path

examination_paper_list = []

path = Path("./docx_markdowns")

for file in path.glob("*.md"):
    with open(file, "r", encoding="utf-8") as f:
        examination_paper_list.append({
            "file_path": str(file),
            "text": one_file_per_process(f.read())
        })
len(examination_paper_list)

0

## 定义测试数据

In [4]:
text = examination_paper_list[800]["text"]
for i in text.splitlines():
    print(i)

IndexError: list index out of range

# 主体函数(检测试卷类型)

In [6]:
def find_questions_and_answer_index(lines: list[str]) -> list[int]:
    """
    获取题目和答案的lines下标
    
    判断行的开始是否为 数字+[. ．]
    """
    indexs = []
    pattern = r"^\d+[\.|\．]"
    for index in range(len(lines)):
        match = re.search(pattern, lines[index])
        if match: 
           indexs.append(index)
    return indexs 

In [ ]:
def check_paper_type(text, answer_keywords=['答案']):
    """
    检查试卷类型
    """
    # 答案出现次数
    answer_count = 0
    # 拆分成行
    lines = text.splitlines()
    # 每道题的索引下标
    question_indexs = find_question_index(lines)
    
    for i,question_index in enumerate(question_indexs):
        if i+1 == len(question_indexs):
            break;
        # 获取当前题的文本
        current_question = "".join(lines[question_index:question_indexs[i+1]])
        if any(keyword in current_question for keyword in answer_keywords):
            answer_count += 1
            
    # Todo: 暂时只完成了检测 "试卷解析 "类型
    if answer_count > len(question_indexs)/2:
        return 1
    
    return 0

In [ ]:
print(check_paper_type(text))

## 暂时用不到的代码

In [ ]:
import re

def find_one_question_index(lines):
    indexs = []
    pattern = r'^(一|二|三|四|五|六|七|八|九|十)'
    for index in range(len(lines)):
        match = re.search(pattern, lines[index])
        if match: 
           indexs.append(index)
            
    return indexs